Important note:
This import is faulty. it results in a positive range of values and distorts the data.

In [1]:
%reset -f

import numpy as np
import pickle
import mne
from mne import io
import joblib
from tqdm import tqdm
import gc

## less verbose output
import logging, sys
logging.disable(sys.maxsize)

Choose parameters

In [1]:
all_subjs = ['8','9','10','11','12','13','14','15','16','17','19','20','21','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38']

all_good_subjects = ['32','35','36','37','38']
all_wake_data_subjects = ['32','33','34','35','38']
all_reref_data_subjects = ['32','33','34','35','38']

list1_as_set = set(all_good_subjects)
intersection = list1_as_set.intersection(all_wake_data_subjects)
all_goodAndGotWakeData_subjects = list(intersection)

toneHieghts = ['650','845','1098','1428','1856','2413','3137','4079','5302']
toneTypes = ['O','T'] # There are A/O/T

sleepstages = ['N2', 'N3', 'Rp', 'Rt']
wakestages = ['Wn']
vigelenceStages = np.concatenate((sleepstages,wakestages))

trialNums = range(0,9)

electrodes=[4,5,10,79,84]

data_path='C:\AnatArzData\Data\\rerefrenced'
data_path1='C:\AnatArzData\Data\RDMs_and_distances_results'
data_path2='C:\AnatArzData\Data\EEG_data_for_analysis'

folderpath_output = 'C:\AnatArzData\Data'
filename_evoked = 'subs_evoked_1sub_reref_mneRerefAverage'
filename_epochs = 'subs_epochs_1sub_reref_mneRerefAverage'

##########################
### chosen parameters
##########################
subs = ['32']
toneHieghts = toneHieghts
toneTypes = toneTypes
sleepstages = sleepstages
wakestages = wakestages
trialNums = trialNums
electrodes = electrodes
##########################

NameError: name 'np' is not defined

In [5]:
## import all data to dictionary with events name
def import_data(subs, toneHieghts, toneTypes, sleepstages,wakestages,trialNums, electrodes):
    alldata_epochs = {}
    alldata_evoked = {}
    for sub_i, sub in tqdm(enumerate(subs)):
        ## evacuate memo
        gc.collect()
        try: del currSub_eeglabEpochs
        except NameError: pass

        ## sleep: get eeglabEpochs for a subject
        subject_setfile_sleep = data_path + '/s_' + sub + '_sleep_referenced.set'
        try: currSub_eeglabEpochs = io.read_epochs_eeglab(subject_setfile_sleep, events=None, event_id=None,  eog=(),verbose=None, uint16_codec=None)
        except FileNotFoundError: continue
        currSub_eeglabEpochs = currSub_eeglabEpochs.set_eeg_reference(ref_channels='average')
        ## arrange all epochs in an dict
        for ss_i, ss in enumerate(sleepstages):
            for toneHi_i, toneHi in enumerate(toneHieghts):
                for toneTy_i, toneTy in enumerate(toneTypes):
                    for trial_i, trial in enumerate(trialNums):
                        try:
                            trial_name = f"{ss}{toneTy}{toneHi}_{trial}"
                            currTrial_eeglabEpoch = currSub_eeglabEpochs[trial_name]
                            currTrial_mneEpoch = mne.concatenate_epochs([currTrial_eeglabEpoch])
                            currTrial_mneEpoch.pick(electrodes)
                            alldata_epochs[f"s{sub}_{trial_name}"] = currTrial_mneEpoch
                            alldata_evoked[f"s{sub}_{trial_name}"] = currTrial_mneEpoch.average()
                        except KeyError: continue

        ## evacuate memo
        gc.collect()
        try: del currSub_eeglabEpochs
        except NameError: pass
        ## Wake: get eeglabEpochs for a subject
        try:
            subject_setfile_wake = data_path + '/s_' + sub + '_wake_night_referenced.set'
            currSub_eeglabEpochs = io.read_epochs_eeglab(subject_setfile_wake, events=None, event_id=None,  eog=(),verbose=None, uint16_codec=None)
        except FileNotFoundError: continue
        ## arrange all epochs in an dict
        for ws_i, ws in enumerate(wakestages):
            for toneHi_i, toneHi in enumerate(toneHieghts):
                for toneTy_i, toneTy in enumerate(toneTypes):
                    for trial_i, trial in enumerate(trialNums):
                        try:
                            if ws == 'Wn':
                                ws_name = ''
                            else:
                                ws_name = ws
                            trial_name = f"{ws_name}{toneTy}{toneHi}_{trial}"
                            currTrial_eeglabEpoch = currSub_eeglabEpochs[trial_name]
                            #currTrial_eeglabEpoch = currTrial_eeglabEpoch.set_eeg_reference(ref_channels='average')
                            currTrial_mneEpoch = mne.concatenate_epochs([currTrial_eeglabEpoch])
                            currTrial_mneEpoch.pick(electrodes)
                            alldata_epochs[f"s{sub}_{ws}{trial_name}"] = currTrial_mneEpoch
                            alldata_evoked[f"s{sub}_{ws}{trial_name}"] = currTrial_mneEpoch.average()
                        except KeyError: continue
    return  alldata_epochs, alldata_evoked

alldata_epochs, alldata_evoked = import_data(subs, toneHieghts, toneTypes, sleepstages,wakestages,trialNums, electrodes)

0it [00:00, ?it/s]C:\Users\Sharon\anaconda3\Lib\site-packages\pymatreader\utils.py:122: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\Sharon\AppData\Local\Temp\ipykernel_22292\1772481130.py:13: RuntimeWarning: Unknown types found, setting as type EEG:
ref: ['Cz']
  try: currSub_eeglabEpochs = io.read_epochs_eeglab(subject_setfile_sleep, events=None, event_id=None,  eog=(),verbose=None, uint16_codec=None)
C:\Users\Sharon\anaconda3\Lib\site-packages\pymatreader\utils.py:122: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\Sharon\AppData\Local\Temp\ipykernel_22292\1772481130.py:37: RuntimeWarning: Unknown types found, setting as type EEG:
ref: ['Cz']
  currSub_eeglabEpochs = io.read_epoch

In [ ]:
# save
joblib.dump(alldata_evoked, f"{folderpath_output}\\{filename_evoked}")
joblib.dump(alldata_epochs, f"{folderpath_output}\\{filename_epochs}")
#alldata_evoked = joblib.load(filename)

In [ ]:
# Saving the objects:
f = open(f"{folderpath_output}\\{filename_epochs}_vars.pkl", 'wb')
pickle.dump([subs, toneHieghts, toneTypes, sleepstages,wakestages,trialNums, electrodes], f)
f.close()

## Try with eeglab seperately improted data

In [ ]:
## import all data to dictionary with events name
def import_data(subs, toneHieghts, toneTypes, sleepstages,wakestages,trialNums, electrodes):
    alldata_epochs = {}
    alldata_evoked = {}
    for sub_i, sub in tqdm(enumerate(subs)):
        ## evacuate memo
        gc.collect()
        try: del currSub_eeglabEpochs
        except NameError: pass

        ## sleep: get eeglabEpochs for a subject
        subject_setfile_sleep = data_path + '/s_' + sub + '_sleep_referenced.set'
        try: currSub_eeglabEpochs = io.read_epochs_eeglab(subject_setfile_sleep, events=None, event_id=None,  eog=(),verbose=None, uint16_codec=None)
        except FileNotFoundError: continue

        currSub_eeglabEpochs.

        ## arrange all epochs in an dict
        for ss_i, ss in enumerate(sleepstages):
            for toneHi_i, toneHi in enumerate(toneHieghts):
                for toneTy_i, toneTy in enumerate(toneTypes):
                    for trial_i, trial in enumerate(trialNums):
                        try:
                            trial_name = f"{ss}{toneTy}{toneHi}_{trial}"
                            currTrial_eeglabEpoch = currSub_eeglabEpochs[trial_name]
                            currTrial_mneEpoch = mne.concatenate_epochs([currTrial_eeglabEpoch])
                            currTrial_mneEpoch.pick(electrodes)
                            alldata_epochs[f"s{sub}_{trial_name}"] = currTrial_mneEpoch
                            alldata_evoked[f"s{sub}_{trial_name}"] = currTrial_mneEpoch.average()
                        except KeyError: continue

        ## evacuate memo
        gc.collect()
        try: del currSub_eeglabEpochs
        except NameError: pass
        ## Wake: get eeglabEpochs for a subject
        try:
            subject_setfile_wake = data_path + '/s_' + sub + '_wake_night_referenced.set'
            currSub_eeglabEpochs = io.read_epochs_eeglab(subject_setfile_wake, events=None, event_id=None,  eog=(),verbose=None, uint16_codec=None)
        except FileNotFoundError: continue
        ## arrange all epochs in an dict
        for ws_i, ws in enumerate(wakestages):
            for toneHi_i, toneHi in enumerate(toneHieghts):
                for toneTy_i, toneTy in enumerate(toneTypes):
                    for trial_i, trial in enumerate(trialNums):
                        try:
                            if ws == 'Wn':
                                ws_name = ''
                            else:
                                ws_name = ws
                            trial_name = f"{ws_name}{toneTy}{toneHi}_{trial}"
                            currTrial_eeglabEpoch = currSub_eeglabEpochs[trial_name]
                            #currTrial_eeglabEpoch = currTrial_eeglabEpoch.set_eeg_reference(ref_channels='average')
                            currTrial_mneEpoch = mne.concatenate_epochs([currTrial_eeglabEpoch])
                            currTrial_mneEpoch.pick(electrodes)
                            alldata_epochs[f"s{sub}_{ws}{trial_name}"] = currTrial_mneEpoch
                            alldata_evoked[f"s{sub}_{ws}{trial_name}"] = currTrial_mneEpoch.average()
                        except KeyError: continue
    return  alldata_epochs, alldata_evoked

alldata_epochs, alldata_evoked = import_data(subs, toneHieghts, toneTypes, sleepstages,wakestages,trialNums, electrodes)